## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 
             'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 
             'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
print(X_df.dtypes)

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train_encoded = pd.get_dummies(X_train, columns=['OverTime'])
X_test_encoded = pd.get_dummies(X_test, columns=['OverTime'])

# Show value counts of OverTime to verify encoding
print(X_train['OverTime'].value_counts())

OverTime
No     780
Yes    322
Name: count, dtype: int64


In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
numeric_columns = X_train_encoded.select_dtypes(include=['int64', 'float64']).columns
scaler.fit(X_train_encoded[numeric_columns])

# Scale the training and testing data
X_train_scaled = X_train_encoded.copy()
X_test_scaled = X_test_encoded.copy()
X_train_scaled[numeric_columns] = scaler.transform(X_train_encoded[numeric_columns])
X_test_scaled[numeric_columns] = scaler.transform(X_test_encoded[numeric_columns])

In [8]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder to the training and testing data
y_train_dept_encoded = dept_encoder.transform(y_train[['Department']])
y_test_dept_encoded = dept_encoder.transform(y_test[['Department']])

In [9]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attr_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder to the training and testing data
y_train_attr_encoded = attr_encoder.transform(y_train[['Attrition']])
y_test_attr_encoded = attr_encoder.transform(y_test[['Attrition']])

## Part 2: Create, Compile, and Train the Model

In [14]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer1 = layers.Dense(units=50, activation='relu')(input_layer)
shared_layer2 = layers.Dense(units=25, activation='relu')(shared_layer1)


In [16]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer

dept_hidden = layers.Dense(units=15, activation='relu')(shared_layer2)

# Create the output layer

dept_output = layers.Dense(units=3, activation='softmax', name='dense_3')(dept_hidden)  

In [17]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_hidden = layers.Dense(units=10, activation='relu')(shared_layer2)


# Create the output layer
attr_output = layers.Dense(units=2, activation='softmax', name='attrition')(attr_hidden)

In [18]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attr_output])

# Compile the model
model.compile(optimizer='adam',
             loss={'dense_3': 'categorical_crossentropy', 
                   'attrition': 'categorical_crossentropy'},
             metrics={'dense_3': 'accuracy',  
                     'attrition': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 50)        │        600 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 25)        │      1,275 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 15)        │        390 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 10)        │        260 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 3)         │         48 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition (Dense)   │ (None, 2)         │         22 │ dense_8[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,595 (10.14 KB)

 Trainable params: 2,595 (10.14 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Train the model
history = model.fit(X_train_scaled,
                   {'dense_3': y_train_dept_encoded,  # Changed from 'department'
                    'attrition': y_train_attr_encoded},
                   epochs=50,
                   batch_size=32,
                   validation_split=0.2)

Epoch 1/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - attrition_accuracy: 0.7694 - attrition_loss: 0.6305 - dense_3_accuracy: 0.5121 - dense_3_loss: 1.0155 - loss: 1.6462 - val_attrition_accuracy: 0.7873 - val_attrition_loss: 0.5731 - val_dense_3_accuracy: 0.6063 - val_dense_3_loss: 0.8693 - val_loss: 1.4443
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_accuracy: 0.8293 - attrition_loss: 0.5164 - dense_3_accuracy: 0.6650 - dense_3_loss: 0.8072 - loss: 1.3240 - val_attrition_accuracy: 0.7873 - val_attrition_loss: 0.5189 - val_dense_3_accuracy: 0.6063 - val_dense_3_loss: 0.8408 - val_loss: 1.3623
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_accuracy: 0.8428 - attrition_loss: 0.4306 - dense_3_accuracy: 0.6671 - dense_3_loss: 0.7723 - loss: 1.2030 - val_attrition_accuracy: 0.7873 - val_attrition_loss: 0.4953 - val_dense_3_accuracy: 0.6063 - val_dense_3_loss: 0.8229 - val_loss: 1.3210
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_accuracy: 0.84

In [21]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled,
                            {'dense_3': y_test_dept_encoded,  # Changed from 'department'
                             'attrition': y_test_attr_encoded},
                            verbose=0)

In [22]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[3]:.4f}")
print(f"Attrition Accuracy: {test_results[4]:.4f}")

Department Accuracy: 0.8505
Attrition Accuracy: 0.6033


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 